# SGPS BioStatistics
## Excercise 2: Getting publically availible data

#### Gene Expression data from Gene Expression Omnibus
+ https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE123336

#### Specifically from this paper
1.	LaRocca D, Barns S, Hicks SD, Brindle A et al. Comparison of serum and saliva miRNAs for identification and characterization of mTBI in adult mixed martial arts fighters. *PLoS One* 2019;14(1):e0207785.


In [15]:
# Import the packages we need
import pandas as pd
import numpy as np
# import GEOparse

In [9]:
# Get the data in
human_expression = pd.read_csv('GSE123336_MMA_CountMatrix.csv.gz', index_col=0, compression='gzip')
human_expression = human_expression.apply(pd.to_numeric, errors='coerce')

In [16]:
# Get the data straight (off the omnibus)
gse = GEOparse.get_GEO(geo="GSE123336", destdir=".")

22-Aug-2025 15:20:44 DEBUG utils - Directory . already exists. Skipping.
22-Aug-2025 15:20:44 INFO GEOparse - Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE123nnn/GSE123336/soft/GSE123336_family.soft.gz to ./GSE123336_family.soft.gz
100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 12.6k/12.6k [00:00<00:00, 119kB/s]
22-Aug-2025 15:20:44 DEBUG downloader - Size validation passed
22-Aug-2025 15:20:44 DEBUG downloader - Moving /var/folders/j6/6pz4l2t10238hsq7sl5n5qww0000gn/T/tmpwlbl8jid to /Users/jpw/My Drive/teaching/2025_Biostatistics/e2_Getting_Data/GSE123336_family.soft.gz
22-Aug-2025 15:20:44 DEBUG downloader - Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE123nnn/GSE123336/soft/GSE123336_family.soft.gz
22-Aug-2025 15:20:44 INFO GEOparse - Parsing ./GSE123336_family.soft.gz: 
22-Aug-2025 15:20:44 DEBUG GEOparse - DATABASE: GeoMiame
22-Aug-2025 15:20:44 DEBUG GEOparse - SERIES: GSE123336
22-Aug-2025 15:20:

In [17]:
gse.gsms

{'GSM3500956': <SAMPLE: GSM3500956>,
 'GSM3500957': <SAMPLE: GSM3500957>,
 'GSM3500958': <SAMPLE: GSM3500958>,
 'GSM3500959': <SAMPLE: GSM3500959>,
 'GSM3500960': <SAMPLE: GSM3500960>,
 'GSM3500961': <SAMPLE: GSM3500961>,
 'GSM3500962': <SAMPLE: GSM3500962>,
 'GSM3500963': <SAMPLE: GSM3500963>,
 'GSM3500964': <SAMPLE: GSM3500964>,
 'GSM3500965': <SAMPLE: GSM3500965>,
 'GSM3500966': <SAMPLE: GSM3500966>,
 'GSM3500967': <SAMPLE: GSM3500967>,
 'GSM3500968': <SAMPLE: GSM3500968>,
 'GSM3500969': <SAMPLE: GSM3500969>,
 'GSM3500970': <SAMPLE: GSM3500970>,
 'GSM3500971': <SAMPLE: GSM3500971>,
 'GSM3500972': <SAMPLE: GSM3500972>,
 'GSM3500973': <SAMPLE: GSM3500973>,
 'GSM3500974': <SAMPLE: GSM3500974>,
 'GSM3500975': <SAMPLE: GSM3500975>,
 'GSM3500976': <SAMPLE: GSM3500976>,
 'GSM3500977': <SAMPLE: GSM3500977>,
 'GSM3500978': <SAMPLE: GSM3500978>,
 'GSM3500979': <SAMPLE: GSM3500979>,
 'GSM3500980': <SAMPLE: GSM3500980>,
 'GSM3500981': <SAMPLE: GSM3500981>,
 'GSM3500982': <SAMPLE: GSM3500982>,
 

In [18]:
gse.gsms['GSM3500956'].metadata

{'title': ['MMA001-2533'],
 'geo_accession': ['GSM3500956'],
 'status': ['Public on Dec 05 2018'],
 'submission_date': ['Dec 04 2018'],
 'last_update_date': ['Dec 05 2018'],
 'type': ['SRA'],
 'channel_count': ['1'],
 'source_name_ch1': ['Saliva'],
 'organism_ch1': ['Homo sapiens'],
 'taxid_ch1': ['9606'],
 'characteristics_ch1': ['tissue: Saliva',
  'timepoint: 0d post',
  'hits to the head: 2',
  'subject: MMA001'],
 'molecule_ch1': ['total RNA'],
 'extract_protocol_ch1': ['Saliva was collected in an RNA stabilizing container (Oragene RNA RE-100, DNA Genotek) and RNA was harvested using Qiazol reagent followed by RNeasy (Qiagen).',
  'llumina TruSeq RNA Sample Prep Kit (Cat#FC-122-1001) was used with 250-500 nanograms of total RNA for the construction of sequencing libraries.'],
 'description': ['Sample 1'],
 'data_processing': ['Raw sequencing intensities were obtained on a NextSeq500 instrument and converted to base calls using bcl2fastq software (Illumina)',
  'TruSeq Small RNA ad

In [19]:
# And collect the metadata
meta = {}
for key in gse.gsms:
    # print(key)
    samp = gse.gsms[key].metadata['description'][0]
    characteristics = {}
    for item in gse.gsms[key].metadata['characteristics_ch1']:
        temp = item.split(': ')
        characteristics[temp[0]] = temp[1]

    if samp not in meta:
        meta[samp] = {}
        meta[samp] = characteristics
    else:
        meta[samp] = characteristics

metadata = pd.DataFrame(meta).T
metadata.to_csv('GSE123336_metadata.csv')

In [20]:
metadata = pd.read_csv('GSE123336_metadata.csv', index_col=0)
metadata

,tissue,timepoint,hits to the head,subject
Sample 1,Saliva,0d post,2,MMA001
Sample 2,Serum,0d post,2,MMA001
Sample 3,Saliva,0d pre,0,MMA001
Sample 4,Serum,0d pre,0,MMA001
Sample 5,Saliva,1wk post,2,MMA001
...,...,...,...,...
Sample 214,Serum,0d pre,0,MMA040
Sample 215,Serum,0d post,35,MMA041
Sample 216,Serum,0d pre,0,MMA041
Sample 217,Serum,0d post,7,MMA042


In [21]:
# Can we see which miRNA are differentially expressed in Saliva pre and post fight?
# Filter for saliva samples only
saliva_samples = metadata[(metadata['tissue'] == 'Saliva') & 
                          (metadata['timepoint'].isin(['0d pre', '0d post']))]

# Get the corresponding sample names
saliva_sample_names = saliva_samples.index

# Subset the expression data
saliva_expression = human_expression[saliva_sample_names]
saliva_expression

pre_fight_samples = saliva_samples[saliva_samples['timepoint'] == '0d pre'].index
post_fight_samples = saliva_samples[saliva_samples['timepoint'] == '0d post'].index

pre_fight_expression = saliva_expression[pre_fight_samples]
post_fight_expression = saliva_expression[post_fight_samples]

In [22]:
from scipy.stats import mannwhitneyu

# Initialize a list to store results
diff_expression_results = []

# Iterate through each miRNA
for miRNA in saliva_expression.index:
    # Extract expression levels for this miRNA
    pre_values = pre_fight_expression.loc[miRNA]
    post_values = post_fight_expression.loc[miRNA]
    
    # Perform the Mann-Whitney U test
    stat, p_value = mannwhitneyu(pre_values, post_values, alternative='two-sided')
    
    # Store the results
    diff_expression_results.append({
        'miRNA': miRNA,
        'statistic': stat,
        'p_value': p_value
    })

# Convert the results to a DataFrame
diff_expression_df = pd.DataFrame(diff_expression_results)

# Apply multiple testing correction (e.g., Benjamini-Hochberg)
from statsmodels.stats.multitest import multipletests

diff_expression_df['Significant'], diff_expression_df['adjusted_p_value'], _, _ = multipletests(diff_expression_df['p_value'], method='fdr_bh')

# Sort the results by adjusted p-value
diff_expression_df = diff_expression_df.sort_values('p_value')
diff_expression_df

,miRNA,statistic,p_value,Significant,adjusted_p_value
1287,hsa-miR-486-3p,96.0,0.001339,False,1.0
351,hsa-miR-204-5p,108.0,0.004007,False,1.0
960,hsa-miR-449c-5p,121.5,0.007006,False,1.0
992,hsa-miR-4524b-3p,132.0,0.010144,False,1.0
47,hsa-miR-1185-2-3p,142.0,0.011817,False,1.0
...,...,...,...,...,...
1992,hsa-miR-6805-3p,208.5,1.000000,False,1.0
547,hsa-miR-3155a,209.5,1.000000,False,1.0
322,hsa-miR-196b-3p,209.0,1.000000,False,1.0
325,hsa-miR-197-5p,208.5,1.000000,False,1.0


In [ ]:
# Can we see which miRNA are differentially expressed in Serum pre and post fight?
# Filter for serum samples only


### How about getting rat data for mTBI?
#### Gene Expression data from Gene Expression Omnibus
+ https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE159011

#### Specifically from this paper
1.	Das Gupta S, Ciszek R, Heiskanen M, Lapinlampi N et al. Plasma miR-9-3p and miR-136-3p as Potential Novel Diagnostic Biomarkers for Experimental and Human Mild Traumatic Brain Injury. *Int J Mol Sci* 2021 Feb 4;22(4).


In [ ]:
rat_expression = pd.read_csv("GSE159011_Raw_counts_matrix.txt", index_col=0, sep='\t')

In [ ]:
rat_expression

In [ ]:
rat_expression.sum()

In [ ]:
# Normalization
# Step 1: Calculate the sum of each column
column_sums = rat_expression.sum()

# Step 2: Determine the normalization factor to scale each column to sum to 1 million
normalization_factor = 1_000_000 / column_sums

# Step 3: Multiply each column by its respective normalization factor
normalized_df = rat_expression.multiply(normalization_factor, axis=1)

# Now each column in `normalized_df` should sum to 1 million
normalized_df

In [ ]:
# We also want to do put this on a log2 scale
log2_df = np.log2(normalized_df + 1)
log2_df

In [23]:
# Get the data straight (off the omnibus)
# gse = GEOparse.get_GEO(geo="GSE159011", destdir=".")

22-Aug-2025 15:21:37 DEBUG utils - Directory . already exists. Skipping.
22-Aug-2025 15:21:37 INFO GEOparse - Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE159nnn/GSE159011/soft/GSE159011_family.soft.gz to ./GSE159011_family.soft.gz
100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 4.47k/4.47k [00:00<00:00, 39.7kB/s]
22-Aug-2025 15:21:37 DEBUG downloader - Size validation passed
22-Aug-2025 15:21:37 DEBUG downloader - Moving /var/folders/j6/6pz4l2t10238hsq7sl5n5qww0000gn/T/tmpkvqpjbab to /Users/jpw/My Drive/teaching/2025_Biostatistics/e2_Getting_Data/GSE159011_family.soft.gz
22-Aug-2025 15:21:37 DEBUG downloader - Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE159nnn/GSE159011/soft/GSE159011_family.soft.gz
22-Aug-2025 15:21:37 INFO GEOparse - Parsing ./GSE159011_family.soft.gz: 
22-Aug-2025 15:21:37 DEBUG GEOparse - DATABASE: GeoMiame
22-Aug-2025 15:21:37 DEBUG GEOparse - SERIES: GSE159011
22-Aug-2025 15:21:

In [24]:
# And collect the metadata
meta = {}
for key in gse.gsms:
    # print(key)
    samp = gse.gsms[key].metadata['description'][0]
    characteristics = {}
    for item in gse.gsms[key].metadata['characteristics_ch1']:
        temp = item.split(': ')
        characteristics[temp[0]] = temp[1]

    if samp not in meta:
        meta[samp] = {}
        meta[samp] = characteristics
    else:
        meta[samp] = characteristics

rat_metadata = pd.DataFrame(meta).T
rat_metadata.to_csv('GSE159011_metadata.csv')

In [ ]:
rat_metadata = pd.read_csv('GSE159011_metadata.csv', index_col=0)
rat_metadata

In [ ]:
# Filter metadata for Naive and Severe TBI groups
log2_df.columns = log2_df.columns.str.replace('-UMIs', '')
naive_samples = rat_metadata[rat_metadata['group'] == 'Naive'].index
severe_tbi_samples = rat_metadata[rat_metadata['group'] == 'Severe TBI'].index

In [ ]:
# Subset expression data for Naive and Severe TBI samples
naive_expression = log2_df[naive_samples]
severe_tbi_expression = log2_df[severe_tbi_samples]

In [ ]:
# Initialize a list to store results
diff_expression_results = []

# Iterate through each miRNA
for miRNA in log2_df.index:
    # Extract expression levels for this miRNA in Naive and Severe TBI groups
    naive_values = naive_expression.loc[miRNA]
    severe_tbi_values = severe_tbi_expression.loc[miRNA]
    
    # Perform the Mann-Whitney U test
    stat, p_value = mannwhitneyu(naive_values, severe_tbi_values, alternative='two-sided')
    
    # Store the results
    diff_expression_results.append({
        'miRNA': miRNA,
        'statistic': stat,
        'p_value': p_value
    })

# Convert the results to a DataFrame
diff_expression_df = pd.DataFrame(diff_expression_results)

# Drop out NaN from the p-values
diff_expression_df = diff_expression_df.dropna(subset=['p_value'])

# Apply multiple testing correction (e.g., Benjamini-Hochberg)
from statsmodels.stats.multitest import multipletests

diff_expression_df['adjusted_p_value'] = multipletests(diff_expression_df['p_value'], method='fdr_bh')[1]

# Apply multiple testing correction (e.g., Benjamini-Hochberg)
from statsmodels.stats.multitest import multipletests

diff_expression_df['Significant'], diff_expression_df['adjusted_p_value'], _, _ = multipletests(diff_expression_df['p_value'], method='fdr_bh')

# Sort the results by adjusted p-value
diff_expression_df = diff_expression_df.sort_values('p_value')
diff_expression_df